# Hand-made Standardizer

## 1. Key Concept : State-less vs. State-full transformers

👇 Consider the following train and test sets

In [1]:
import numpy as np
import pandas as pd

X_train = pd.DataFrame({
    'A': {0: 1, 1: 2, 2: 3},
    'B': {0: 2, 1: 3, 2: 4},
    'C': {0: 3, 1: 4, 2: 5}})
display(X_train)

X_test = pd.DataFrame({
    'A': {0: 1, 1: 2, 2: 3},
    'B': {0: 2, 1: 3, 2: 4},
    'C': {0: 3, 1: 4, 2: 10}})
display(X_test)

,A,B,C
0,1,2,3
1,2,3,4
2,3,4,5


,A,B,C
0,1,2,3
1,2,3,4
2,3,4,10


👇 And the following pipeline

In [2]:
from sklearn import set_config; set_config(display='diagram')
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.pipeline import make_pipeline, make_union

scaler = StandardScaler()
feature_averager = FunctionTransformer(lambda df: pd.DataFrame(1/3 * (df["A"] + df["B"] + df["C"])))
pipe = make_union(scaler, feature_averager)
pipe

FeatureUnion(transformer_list=[('standardscaler', StandardScaler()),
                               ('functiontransformer',
                                FunctionTransformer(func=<function <lambda> at 0x7efca5dc9b80>))])

In [3]:
pipe.fit(X_train)
pd.DataFrame(pipe.transform(X_train))

,0,1,2,3
0,-1.224745,-1.224745,-1.224745,2.0
1,0.000000,0.000000,0.000000,3.0
2,1.224745,1.224745,1.224745,4.0


In [4]:
pd.DataFrame(pipe.transform(X_test))

,0,1,2,3
0,-1.224745,-1.224745,-1.224745,2.000000
1,0.000000,0.000000,0.000000,3.000000
2,1.224745,1.224745,7.348469,5.666667


☝️ Notice how the `StandardScaler` and the `FunctionTransformer` are fundamentally different:

1️⃣ `FunctionTransformer` can only performs **stateless** transformations
 
$(X_1, X_2, X_3)$ --> $\frac{(X_1 + X_2 + X_3)}{3}$ for our `feature_averager`

other stateless transformations for instance: 

$X$ --> $log(X)$  
$(X_1, X_2)$ --> $X_1 + 5X_2^2$ 

2️⃣ `StandardScaler` performs a **state-full** transformation 

$
X \rightarrow \frac{(X-\mu )}{\sigma}
$

- that requires to **store** information from the train set during the `.fit` (here, `mean_train` and `std_train`)
- In order to **reuse/apply** these back later during the `.transform` phase, on *both* train or test sets

☝️ What if we wanted to code our own state-full custom transformer? For that, we will have to code our own class

## 2. Create your own state-full transformer

### 2.1 CustomStandardizer

👉 Try to code your own class `CustomStandardizer` that should behave exactly like `StandardScaler` from scikit-learn.  
This means having a `fit()` and `transform()` method.

Then, fit it on `X_train` and transform `X_test` with it to compare with the original scikit-learn version!





In [5]:
# TransformerMixin inheritance is used to create fit_transform() method from fit() and transform()
from sklearn.base import TransformerMixin, BaseEstimator



class CustomStandardizer(TransformerMixin, BaseEstimator):
    
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        # Store what needs to be stored as instance attributes. Return "self" to allow chaining fit and transform.
        self.means = X.mean()
        self.stds = X.std(ddof=0)
        return self
    
    def transform(self, X, y=None): 
        X_new = (X - self.means) / self.stds
        return X_new
    
    

In [6]:
# Try it out below
custom_standardizer = CustomStandardizer()
custom_standardizer.fit(X_train)
custom_standardizer.transform(X_test)

,A,B,C
0,-1.224745,-1.224745,-1.224745
1,0.000000,0.000000,0.000000
2,1.224745,1.224745,7.348469


In [7]:
from nbresult import ChallengeResult

tmp = CustomStandardizer()
tmp_train = np.array(tmp.fit_transform(X_train))
tmp_test = np.array(tmp.transform(X_test))

result = ChallengeResult('standardizer', 
                         X_train_transformed=tmp_train,
                         X_test_transformed=tmp_test
)

result.write()
print(result.check())

============================= test session starts ==============================
platform linux -- Python 3.8.12, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /home/matheus/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /home/matheus/code/matheussposito/data-challenges-869/05-ML/08-Workflow/05-Hand-Made-Standardizer
plugins: anyio-3.4.0
collecting ... collected 1 item

tests/test_standardizer.py::TestStandardizer::test_solution PASSED       [100%]

============================== 1 passed in 0.21s ===============================


💯 You can commit your code:

git add tests/standardizer.pickle

git commit -m 'Completed standardizer step'

git push origin master


<details>
<summary>💡 Hint if test above only fail by a small margin </summary>

Be carefull there is a slight difference between `np.std()` and `pd.std` methods! This stackoverflow [post](https://stackoverflow.com/questions/44220290/sklearn-standardscaler-result-different-to-manual-result) might help 😉
      
</details>

### 2.2 Inverse Transform

❗️ Scikit-learn transformer also have [`inverse_transform`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler.inverse_transform) methods. Try to implement it in your custom scaler!

In [8]:
# TransformerMixin inheritance is used to create fit_transform() method from fit() and transform()
from sklearn.base import TransformerMixin, BaseEstimator


class CustomStandardizer(TransformerMixin, BaseEstimator):
    
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        # Store what needs to be stored as instance attributes. Return "self" to allow chaining fit and transform.
        self.means = X.mean()
        self.stds = X.std(ddof=0)
        return self
    
    def transform(self, X, y=None): 
        X_new = (X - self.means) / self.stds
        return X_new
    
    def inverse_transform(self, X, y=None):
        X_inv = X * self.stds + self.means
        return X_inv

In [9]:
# Test yourself below

custom_scaler = CustomStandardizer().fit(X_train)
X_train_transformed = custom_scaler.transform(X_train)
display(X_train_transformed)

X_train_detransformed = custom_scaler.inverse_transform(X_train_transformed)
display(X_train_detransformed)

,A,B,C
0,-1.224745,-1.224745,-1.224745
1,0.000000,0.000000,0.000000
2,1.224745,1.224745,1.224745


,A,B,C
0,1.0,2.0,3.0
1,2.0,3.0,4.0
2,3.0,4.0,5.0


In [10]:
assert np.allclose(X_train_detransformed, X_train)

### 2.3 Complete custom pipeline!

👉 Now that we have replicated scikit-learn's `StandardScaler`, we create many new ones !

Try to create the following:

- A `CustomStandardizer(shrink_factor = 1)` which takes one additional argument to allow scaling by more than 1 standard deviation


- A `FeatureAverager()` class that improves upon the one you built on section 1, scaling the result of the 

$$(X_1, X_2, X_3) --> \frac{1/3 * (X_1 + X_2 + X_3)}{max(X_1, X_2, X_3)}$$

Then, use them both to your ininitial feature union `pipe` to make your own custom pipeline!

In [11]:
# Custom Standardizer

class CustomStandardizer(TransformerMixin, BaseEstimator):
    
    def __init__(self, shrink=1):
        self.shrink = shrink
    
    def fit(self, X, y=None):
        # Store what needs to be stored as instance attributes. Return "self" to allow chaining fit and transform.
        self.means = X.mean()
        self.stds = X.std(ddof=0)
        return self
    
    def transform(self, X, y=None): 
        X_new = (X - self.means) / (self.stds * self.shrink)
        return X_new
    
    def inverse_transform(self, X, y=None):
        X_inv = X * (self.stds * self.shrink) + self.means
        return X_inv

Test you new `CustomStandardizer` custom transformer by fitting on `X_train` and transforming it

In [12]:
custom_scaler = CustomStandardizer(shrink=2).fit(X_train)

X_train_transformed = custom_scaler.transform(X_train)
display(X_train_transformed)

X_train_detransformed = custom_scaler.inverse_transform(X_train_transformed)
display(X_train_detransformed)

,A,B,C
0,-0.612372,-0.612372,-0.612372
1,0.000000,0.000000,0.000000
2,0.612372,0.612372,0.612372


,A,B,C
0,1.0,2.0,3.0
1,2.0,3.0,4.0
2,3.0,4.0,5.0


In [13]:
#(X_train['A'] + X_train['B'] + X_train['C']).max()

(1/3 * X_train['A'] + X_train['B'] + X_train['C'])/12

0    0.444444
1    0.638889
2    0.833333
dtype: float64

In [14]:
X_train['A'] + X_train['B'] + X_train['C']

0     6
1     9
2    12
dtype: int64

In [15]:
X_train.sum()

A     6
B     9
C    12
dtype: int64

In [16]:
# Feature Averager

class FeatureAverager(TransformerMixin, BaseEstimator):
    
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        # Store what needs to be stored as instance attributes. Return "self" to allow chaining fit and transform.
        self.features_sum = X['A'] + X['B'] + X['C']
        self.max_factor = np.max(self.features_sum)
        # Return self to allow chaining & fit_transform
        return self
    
    def transform(self, X, y=None): 
        feature_averager = (1/3 * self.features_sum) / self.max_factor
        return pd.DataFrame(feature_averager)

Test you `FeatureAverager` custom transformer by fitting on `X_train` and transforming it

In [17]:
custom_feature_averager = FeatureAverager().fit(X_train)

X_train_transformed = custom_feature_averager.transform(X_train)
display(X_train_transformed)

,0
0,0.166667
1,0.250000
2,0.333333


In [18]:
X_test_transformed = custom_feature_averager.transform(X_test)
display(X_test_transformed)

,0
0,0.166667
1,0.250000
2,0.333333


In [19]:
from nbresult import ChallengeResult

tmp = FeatureAverager()
tmp_train = np.array(tmp.fit_transform(X_train))
tmp_test = np.array(tmp.transform(X_test))

result = ChallengeResult('feature_averager', 
                         X_train_transformed=tmp_train,
                         X_test_transformed=tmp_test
)

result.write()
print(result.check())

============================= test session starts ==============================
platform linux -- Python 3.8.12, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /home/matheus/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /home/matheus/code/matheussposito/data-challenges-869/05-ML/08-Workflow/05-Hand-Made-Standardizer
plugins: anyio-3.4.0
collecting ... collected 1 item

tests/test_feature_averager.py::TestFeatureAverager::test_solution PASSED [100%]

============================== 1 passed in 0.21s ===============================


💯 You can commit your code:

git add tests/feature_averager.pickle

git commit -m 'Completed feature_averager step'

git push origin master


Create a feature union named `pipe` using your custom standardizer and the feature averager created

In [20]:
custom_standardizer = CustomStandardizer(shrink=1)
custom_feature_averager = FeatureAverager()

pipe = make_union(custom_standardizer, custom_feature_averager)
pipe

FeatureUnion(transformer_list=[('customstandardizer', CustomStandardizer()),
                               ('featureaverager', FeatureAverager())])

Fit the `X_train` and `X_test` and transform them

In [21]:
pipe.fit(X_train)
pd.DataFrame(pipe.transform(X_train))

,0,1,2,3
0,-1.224745,-1.224745,-1.224745,0.166667
1,0.000000,0.000000,0.000000,0.250000
2,1.224745,1.224745,1.224745,0.333333


In [22]:
pd.DataFrame(pipe.transform(X_test))

,0,1,2,3
0,-1.224745,-1.224745,-1.224745,0.166667
1,0.000000,0.000000,0.000000,0.250000
2,1.224745,1.224745,7.348469,0.333333


In [23]:
from nbresult import ChallengeResult

tmp = pipe
tmp_train = np.array(tmp.fit_transform(X_train))
tmp_test = np.array(tmp.transform(X_test))

result = ChallengeResult('feature_union_custom_transformers', 
                         X_train_transformed=tmp_train,
                         X_test_transformed=tmp_test
)

result.write()
print(result.check())

============================= test session starts ==============================
platform linux -- Python 3.8.12, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /home/matheus/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /home/matheus/code/matheussposito/data-challenges-869/05-ML/08-Workflow/05-Hand-Made-Standardizer
plugins: anyio-3.4.0
collecting ... collected 1 item

tests/test_feature_union_custom_transformers.py::TestFeatureUnionCustomTransformers::test_solution PASSED [100%]

============================== 1 passed in 0.18s ===============================


💯 You can commit your code:

git add tests/feature_union_custom_transformers.pickle

git commit -m 'Completed feature_union_custom_transformers step'

git push origin master


🏁 Congratulation! Don't forget to commit and push your notebooks